# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
# импорт библиотеки pandas
import pandas as pd

# импорт библиотеки pymystem3
from pymystem3 import Mystem

# импорт контейнера Counter из модуля collections
from collections import Counter

# сохранение Mystem() в m
m = Mystem()

# чтение файла с данными с сохранением в df
df = pd.read_csv('/datasets/data.csv')

# получение первых 10 строк таблицы df
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


    Рассмотрев таблицу видим, что каждая строка таблицы содержит информацию о заёмщиках кредитов в банке: 
    children — количество детей в семье
    days_employed — общий трудовой стаж в днях
    dob_years — возраст клиента в годах
    education — уровень образования клиента
    education_id — идентификатор уровня образования
    family_status — семейное положение
    family_status_id — идентификатор семейного положения
    gender — пол клиента
    income_type — тип занятости
    debt — имел ли задолженность по возврату кредитов
    total_income — ежемесячный доход
    purpose — цель получения кредита

Изучим общую информацию о таблице. Для этого используем метод info().

In [2]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрев полученную информацию о данных в таблице видим, что количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения. В столбце education данные записаны в разных регистрах, а в столбцах days_employed и  total_income тип данных вещественный. 

Проверим данные таблицы на наличие артефактов.

Получим список всех значений в столбце children. Для этого используем метод value_counts(). 

In [3]:
# получение значений столбца children 
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Мы обнаружили два артефакта в количестве детей: 20 и -1.  

Получим список всех уникальных значений в столбце dob_years. Для этого используем метод unique(). 

In [4]:
# получение списка уникальных значений столбца dob_years
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

Мы обнаружили артефакт: 0 лет.  

Получим количество строк в которых значение dob_years равно 0.

In [5]:
# получение значений столбца dob_years равных нулю 
df[df['dob_years'] == 0]['dob_years'].count()

101

В столбце days_employed мы видели, что встречаются данные с отрицательным значением. Проанализируем отрицательные зачения трудового стажа в разрезе типов занятости. Для этого применим логическое условие к столбцу days_employed и метод unique() к столбцу income_type.

In [6]:
# определение типов занятости в которых присутвуют положительные значения
df[df['days_employed'] > 0]['income_type'].unique()

array(['пенсионер', 'безработный'], dtype=object)

Только у категорий 'пенсионер' и 'безработный' положительные значения стажа.

Проверим корректно ли были внесены данные о стаже. Для этого определим максимальное количество дней стажа и переведем это значение в года.

In [7]:
# определение максимального количества дней трудового стажа
days_max = df['days_employed'].max() 

In [8]:
# перевод дней в года
days_max / 365

1100.6997273296713

**Вывод**

Предположения подтвердились. В данных действительно присутствуют артефакты. В столбце children были обнаружены отрицательные значения и значения очень большого количества детей. Это могло произойти в результате опечатки специалиста, который вносил данные. Такие артефакты могут повлиять на результаты исследования.

Также выяснилось, что отрицательный стаж в данных у всех категорий, за исключением пенсионеров и безработных. Это может быть связано с тем, что у пенсионеров и безработных есть дата окончания трудовой деятельности, а остальные категории продолжают работать, либо учиться и даты окончания деятельности у них нет. Также в столбце days_employed мы проверили максимальное количество дней трудового стажа и перевели это значение в года. Получилось, что максимальный стаж составляет 1100 лет. Причиной этому может послужить то, что данные вводились не в днях, а в других единицах измерения. Такие данные являются артефактами. В наших расчетах не будут задействованы данные трудового стажа, поэтому мы не будем их обрабатывать, но для наглядности исправим отрицательные значения стажа. 

В столбце dob_years присутвуют артефакты: в 101 строке стоит значение 0.

Проблемы, которые нужно решать: пропуски в данных, приведение к единому регистру данных об образовании, обработка артефактов в данных о количестве детей и в возрасте заемщиков, замена значений о трудовом стаже и ежемесячном доходе на целочисленный тип данных.     

Для проверки рабочих гипотез особенно ценны столбцы children, family_status, total_income, purpose и debt. 

## Шаг 2. Предобработка данных

### Обработка пропусков

Обработаем артефакты в столбце 'children'. Для этого используем метод replace(), чтобы заменить 20 на 2, а -1 на 1. 

In [9]:
# замена значения в столбце children методом replace()
df.loc[df['children'] == 20, 'children'] = df.loc[df['children'] == 20, 'children'].replace(20, 2)

In [10]:
# замена отрицательного значения в столбце children методом replace()
df.loc[df['children'] == -1, 'children'] = df.loc[df['children'] == -1, 'children'].replace(-1, 1)

In [11]:
# проверка результата
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Избавимся от отрицательного значения стажа с помощью метода abs().

In [12]:
# взятие значений столбца 'days_employed' по модулю 
df['days_employed'] = df['days_employed'].abs()

In [13]:
# проверим строки с отрицательным значением
df[df['days_employed'] < 0].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Проверим данные на наличие пропусков вызовом методов для суммирования пропущенных значений.

In [14]:
# суммарное количество пропусков, выявленных методом isnull() в таблице df
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Видно, что в столбцах days_employed и total_income пропущено одинаковое количество значений. Это означает, что если у клиента отсутствуют данные о трудовом стаже, то у него также отсутствуют данные о ежемесячном доходе. Причины пропусков могут быть разные: возможно клиент не предоставил данные о трудовом стаже и уровне дохода.

Общее количество пропущенных значений 2174 - это 10,1% от общего количества записей. Удалять такое количество данных нельзя, так как это может существенно повлиять на результаты исследования.

Данные в столбцах days_employed и total_income относятся к количественным переменным. Данные из столбца days_employed не фигурируют ни в одной из гипотез, поэтому пропуски заполним нулями. 

In [15]:
# замена пропущенных значений методом fillna() в столбце days_employed
df['days_employed'] = df['days_employed'].fillna(0)

In [16]:
# проверка наличия пропусков в столбце days_employed 
df.isnull().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

А вот пропуски в столбце total_income нельзя заполнить нулевыми значениями, так как от размера ежемесячного дохода может зависить способность возврата кредита в срок. Лучше заполнять такие пропуски средним значением или медианой. Воспользуемся медианой, чтобы исключить влияние артефактов на уровень дохода.

In [17]:
# группировка данных по типам занятости методом groupby() и расчет медианы
income_type_median = df.groupby('income_type')['total_income'].median()

In [18]:
#полученные данные
income_type_median 

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [19]:
# заполнение пропусков в столбце total_income медианой по каждому типу занятости
for index in income_type_median.index:
    df.loc[(df['total_income'].isna()) & (df['income_type'] == index), 'total_income'] = income_type_median[index]

In [20]:
# проверка талбицы df на наличие пропусков
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Обработаем артефакты в столбце 'dob_years' по тому же принципу, что и в столбце total_income. Заменим нули медианой по типам занятости.

In [21]:
# группировка данных по типам занятости методом groupby() и расчет медианы
dob_years_median = df.groupby('income_type')['dob_years'].median()

In [22]:
#полученные данные
dob_years_median

income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: dob_years, dtype: float64

In [23]:
# заполнение пропусков в столбце dob_years медианой по каждому типу занятости
for index in dob_years_median.index:
    df.loc[(df['dob_years'] == 0) & (df['income_type'] == index), 'dob_years'] = dob_years_median[index].astype('int')

In [24]:
# проверка на нулевые значения
df[df['dob_years'] == 0]['dob_years'].count()

0

**Вывод**

Мы исправили арфтефакты в данных о количестве детей и о возрасте заемщиков, обработали отрицательные значение трудового стажа и обработали пропуски во всех столбцах.

### Замена типа данных

В таблице в столбцах days_employed и total_income данные относят к типу float, что не удобно для наглядного отражения и может привести к некорректному отображению целочисленных данных. Приведем данных в столбцах total_income и days_employed к целочисленному типу данных.

In [25]:
# замена типа данных в столбе total_income на целочисленный
df['total_income'] = df['total_income'].astype('int')

In [26]:
# проверка результата
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


In [27]:
# замена типа данных в столбе days_employed на целочисленный
df['days_employed'] = df['days_employed'].astype('int')

In [28]:
# проверка результата
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Все данные в таблице имеют целочисленный тип данных.

### Обработка дубликатов

Проверим данные в таблице на наличие грубых дубликатов. Для этого используем метод duplicated().sum().

In [29]:
# получение суммарного количества дубликатов в таблице df
df.duplicated().sum()

54

В нашей таблице их выявилось 54 строки. Грубые или явные дубликаты появляются в результате различных сбоев в программах. Эти строки необходимо удалить. Для удаления используем метод drop_duplicates(). Этот метод используем в связке с методом reset_index() для замены индексов после удаления строк.

In [30]:
# удаление грубых дубликатов методом drop_duplicates() в связке с методом reset_index()
df = df.drop_duplicates().reset_index(drop=True)

In [31]:
# проверка количества дубликатов
df.duplicated().sum()

0

В столбце education данные записаны разным регистром, например: "среднее", "Среднее" и "СРЕДНЕЕ". Для того чтобы выявить дубликаты в таких строках, нужно все данные привести к нижнему регистру. Причинами появления таких дубликатов обычно является человеческих фактор.

In [32]:
# приведение данных в столбце education к нижнему регистру методом str.lower()
df['education'] = df['education'].str.lower()

In [33]:
# проверка количества дубликатов
df.duplicated().sum()

17

После приведения строк к нижнему регистру выявилось еще 17 дубликатов. Их также необходимо удалить.

In [34]:
# удаление дубликатов
df = df.drop_duplicates().reset_index(drop=True)

In [35]:
# проверка количества дубликатов
df.duplicated().sum()

0

Далее мы проверим столбцы family_status, gender и income_type на наличие неявных дубликатов. Для используем метод unique(), который возвращает перечень всех уникальных значений в столбце.

In [36]:
# поиск дубликатов в столбце family_status методом unique() 
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

В столбце family_status есть данные, записанные в разном регистре: 'Не женат / не замужем', их необходимо привести к нижнему регистру. 

In [37]:
# приведение данных в столбце family_status к нижнему регистру методом str.lower()
df['family_status'] = df['family_status'].str.lower()

In [38]:
# поиск дубликатов в столбце gender 
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

В столбце gender обнаружился артефакт. Помимо значений 'F' для обозначения женского пола и 'M' для обозначения лиц мужского пола, выявился неизвестный 'XNA'. Проверим количество строк с таким значением.

In [39]:
# подсчет количества строк со значением XNA в столбце gender
df[df['gender'] == 'XNA']['gender'].count()

1

Оказалось, что это всего одна строка. Удаление одной строки не повлияет на результаты исследований.

In [40]:
# определение индекса строки со значением XNA в столбце gender
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10684,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


In [41]:
# удаление строки со значением XNA в столбце gender
df = df.drop([10684])

In [42]:
# проверка уникальных значений в столбце gender
df['gender'].unique()

array(['F', 'M'], dtype=object)

In [43]:
# поиск дубликатов в столбце income_type методом unique() 
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

В столбце income_type не оказалось каких-либо артефактов или дубликатов.  

**Вывод**

Мы избавились от всех дубликатов в таблице.

### Лемматизация

Цели получения кредита клиенты указывают самостоятельно, а это значит, что одни и те же цели могут быть записаны по разному. Проверим список уникальных значений в столбце purpose.

In [44]:
# получение уникальных значений столбца purpose 
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

При изучении столбца purpose можно увидеть, что список уникальных значений очень большой. Предположение подтвердилось. Одни и те же цели получения кредита описаны по-разному: "покупка жилья", "покупка недвижимости", "строительство жилой недвижимости", "покупка своего жилья" и т.п. Сравнение таких данных невозможно. Для того, чтобы привести значения к небольшому количеству категорий нужно применить лемматизацию исходных данных. 

Напишем функцию, которая на входе будет принимать данные столбца purpose таблицы, проводить лемматизацию, т.е. приведение слов к их словарной форме. Для этого используем метод m.lemmatize(). Для замены данных новыми категориями создадим внутри функции список из 5 категорий, которые будут записаны в их словарной форме: 'свадьба', 'недвижимость', 'жилье', 'образование', 'автомобиль'. Функция будет сравнивать данные после лемматизации со значением из списка новых категорий и возвращать новую обобщенную категорию. Также категория жилье будет заменена категорией недвижимость, так как они являются схожими.

In [45]:
# функция 
def new_purpose(purpose):
    purposes = ['свадьба', 'недвижимость', 'жилье', 'образование', 'автомобиль']
    lemmas = m.lemmatize(purpose)
    for element in purposes:
        if element in lemmas:
            if element == 'жилье':
                return 'недвижимость'
            return element
    
    return 'другое' 

In [46]:
# вызов функции
df['purpose'] = df['purpose'].apply(new_purpose)

In [47]:
# проверка результата
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость


**Вывод**

Произведена лемматизация значений столбца purpose. Данные в столбце заменены на новые обощенные категории, по которым удобно проводить анализ.  

### Категоризация данных

Используем категоризацию данных для проведения сравнения значений в ходе исследования. 

    В исследовании нам нужно будет проверить 4 гипотезы:
    - есть ли зависимость между наличием детей и возвратом кредита в срок?
    - есть ли зависимость между семейным положением и возвратом кредита в срок?
    - есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    - как разные цели кредита влияют на его возврат в срок?

Для проверки 2 и 4 гипотезы уже проведена категоризация данных. В столбце family_status определены категории отнесения клиентов в соответствии с их семейным положением, а в столбце purpose обозначены категория получения целей кредита. Для проверки 1 и 3 гипотезы поведем категоризацию клиентов по наличию детей и по уровню дохода.

1. Категоризация клиентов по наличию детей.

Разделим всех заемщиков на категории 'есть дети' и 'нет детей'.
Для этого напишем функцию, которая будет принимать информацию о количестве детей из таблицы df и в зависимости от результата возвращать категорию.  

In [48]:
# функция
def set_shildren_level(children):
    if children >= 1:
        return 'есть дети' 
    return 'нет детей' 

In [49]:
# категоризация по количеству детей
df['children_category'] = df['children'].apply(set_shildren_level)

In [50]:
# проверка результата
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,есть дети
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,есть дети
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,есть дети
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,нет детей


2. Категоризация клиентов по уровню дохода.

    Разделим всех заемщиков на категории по уровню дохода. Для этого найдем максимальный и минимальный доход и обозначим категории в процентом соотношении от полученного диапазона: 
    высокий - свыше 75%
    выше среднего - от 50% до 75%
    ниже среднего - от 25% до 50%
    низкий - ниже 25%

In [51]:
# максимальный доход в столбце 'total_income' таблицы
max_income = df['total_income'].max()

In [52]:
# минимальный доход в столбце 'total_income' таблицы
min_income = df['total_income'].min()

In [53]:
# установление значения высокого дохода
high_income = (max_income - min_income) * 0.75

In [54]:
# установление значения среднего дохода
medium_income = (max_income - min_income) * 0.5

In [55]:
# установление значения низкого дохода
low_income = (max_income - min_income) * 0.25

Напишем функцию, которая будет принимать информацию о размере ежемесячного дохода из таблицы df и в зависмости от результата возвращать категорию.  

In [56]:
# функция
def set_income_level(income):
    if income <= low_income:
        return 'низкий' 
    if income <= medium_income:
        return 'ниже среднего'
    if income <= high_income:
        return 'выше среднего'    
    return 'высокий' 

In [57]:
# категоризация по уровню дохода
df['income_level'] = df['total_income'].apply(set_income_level)

In [58]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_category,income_level
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,есть дети,низкий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,есть дети,низкий
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,нет детей,низкий
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,есть дети,низкий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,нет детей,низкий


3. Дополнительно проведем категоризацию клиентов по возрасту.

Разделим всех заемщиков на категории по возрасту. Для этого найдем максимальный и минимальный возраст.

In [59]:
# максимальный возраст в столбце 'dob_years' таблицы
max_dob_years = df['dob_years'].max()

In [60]:
max_dob_years

75

In [61]:
# минимальный возраст в столбце 'dob_years' таблицы
min_dob_years = df['dob_years'].min()

In [62]:
min_dob_years 

19

    Обозначим категории в процентом соотношении от полученного диапазона: 
    молодые - до 30 лет
    взрослые - от 30 до 64 лет
    пенсионеры - старше 64 

In [63]:
# функция
def set_dob_years_cat(dob_years):
    if 18 < dob_years < 30:
        return 'молодые' 
    if 29 < dob_years < 64:
        return 'взрослые'        
    return 'пенсионеры' 

In [64]:
# категоризация по возрасту
df['dob_years_cat'] = df['dob_years'].apply(set_dob_years_cat)

In [65]:
# проверка результата
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_category,income_level,dob_years_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,есть дети,низкий,взрослые
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,есть дети,низкий,взрослые
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,нет детей,низкий,взрослые
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,есть дети,низкий,взрослые
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,нет детей,низкий,взрослые


**Вывод**

В таблице определены категории наличия детей, семейного положения, уровня дохода, целей получения кредита и возраста заемщиков. Можно приступать к проверке гипотез.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Выдвинем гипотезу, что при наличии детей у заемщиков повышается риск невозврата кредита в срок. Для проверки гипотезы рассчитаем процент невозврата кредита по категориям наличия детей у заемщиков.

Напишем функцию, которая будет принимать таблицу, категорию по которой нужно сгруппировать данные, столбец с данными о невозвратах кредита и столбец по которому посчитаем всего количество взятых кредитов. Получившие данные умножим на 100, чтобы вывести %. 

In [66]:
# функция
def calc_percent(data, col_1, col_2):
    result = (data.groupby(col_1)[col_2].sum()) / (data[col_1].value_counts()) * 100
    return result 

In [67]:
# сохраним результат работы работы функции в переменной children_debt
children_debt = calc_percent(df, 'children_category', 'debt')

In [68]:
# создадим объект DataFrame с новым именем столбца
children_debt = children_debt.reset_index(name='percentage_of_cases_of_debt')

In [69]:
# для наглядности округлим данные с точностью до одного знака после запятой
children_debt = children_debt.round(1)

In [70]:
# переименуем столбцы
children_debt = children_debt.rename(columns={'index':'children_category', 
                                              'percentage_of_cases_of_debt':'percentage_of_cases_of_debt'})

Посмотрим на получившийся результат.

In [71]:
children_debt

,children_category,percentage_of_cases_of_debt
0,есть дети,9.2
1,нет детей,7.5


Для наглядности выведем сводную таблицу, в которой отразим количество должников и заемщиков по категориям и процент невозврата кредита. 

In [72]:
# создание сводной таблицы методом pivot_table()
pivot_table_child = df.pivot_table(index = 'children_category', columns = 'debt', values = 'children',
                                   aggfunc = 'count', fill_value=0, margins=True)

In [73]:
# переименование колонок
pivot_table_child = pivot_table_child.rename(columns={'debt': 'debt', 0: 'no_debt', 1: 'debt', 'All': 'total_borrowers'})

In [74]:
# соединение таблицы с процентами со сводной таблицей методом merge() 
pivot_table_child_percent = pivot_table_child.merge(children_debt, on='children_category', how='right')

In [75]:
# вывод результатов
pivot_table_child_percent

,children_category,no_debt,debt,total_borrowers,percentage_of_cases_of_debt
0,есть дети,6685,678,7363,9.2
1,нет детей,13027,1063,14090,7.5


**Вывод**

Гипотеза подтвердилась. У заемщиков с детьми процент невозврата кредита в срок составляет 9%, в то время как заемщики не имеющие детей не возвращают кредит только в 7% случаев. Это связано с тем, что заемщикам с детьми нужно содержать не только себя, но и ребенка, а это дополнительная статья расхода.

Мы также можем наблюдать, что заемщиков у которых нет детей почти в 2 раза больше заемщиков с детьми.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Выдвинем гипотезу, что женатые люди чаще возвращают кредиты, чем неженатые. Для проверки гипотезы рассчитаем процент невозврата кредита по семейному положению заемщиков.

In [76]:
# сохраним результат работы работы функции в переменной family_debt
family_debt = calc_percent(df, 'family_status', 'debt')

In [77]:
# создадим объект DataFrame с новым именем столбца
family_debt = family_debt.reset_index(name='percentage_of_cases_of_debt')

In [78]:
# для наглядности округлим данные с точностью до одного знака после запятой
family_debt = family_debt.round(1)

In [79]:
# переименуем столбцы
family_debt = family_debt.rename(columns={'index':'family_status', 
                                              'percentage_of_cases_of_debt':'percentage_of_cases_of_debt'})

Посмотрим на получившийся результат. Отсортируем в порядке убывания процентов.

In [80]:
family_debt.sort_values(by='percentage_of_cases_of_debt', ascending=False)

,family_status,percentage_of_cases_of_debt
4,не женат / не замужем,9.8
2,гражданский брак,9.3
3,женат / замужем,7.5
0,в разводе,7.1
1,вдовец / вдова,6.6


Заемщики в категории 'не женат / не замужем' 'гражданский брак' это чаще всего молодые люди до 30 лет, а в категории 'вдовец / вдова' пенсионеры. Посмотрим есть ли зависимость возврата кредита от возраста заемщиков. Для этого рассчитаем процент невозврата кредита по возрасту заемщиков.

In [81]:
# сохраним результат работы работы функции в переменной years_debt
years_debt = calc_percent(df, 'dob_years_cat', 'debt')

In [82]:
# создадим объект DataFrame с новым именем столбца
years_debt = years_debt.reset_index(name='percentage_of_cases_of_debt')

In [83]:
# для наглядности округлим данные с точностью до одного знака после запятой
years_debt = years_debt.round(1)

In [84]:
# переименуем столбцы
years_debt = years_debt.rename(columns={'index':'dob_years_cat', 
                                              'percentage_of_cases_of_debt':'percentage_of_cases_of_debt'})

Посмотрим на получившийся результат. Отсортируем в порядке убывания процентов.

In [85]:
years_debt.sort_values(by='percentage_of_cases_of_debt', ascending=False)

,dob_years_cat,percentage_of_cases_of_debt
1,молодые,11.0
0,взрослые,7.8
2,пенсионеры,5.3


Действительно зависимость возврата кредита от возраста заемщиков существует. Молодые люди менее ответсвенны и в 11% случаев не возвращают кредиты, в то время как пенсионеры и взрослые люди подходят к этому вопросу более ответственно. 

Для наглядности выведем сводную таблицу, в которой отразим количество должников и заемщиков по категориям и процент невозврата кредита. 

In [86]:
# создание сводной таблицы методом pivot_table()
pivot_table_family = df.pivot_table(index = 'family_status', columns = 'debt', values = 'children',
                                   aggfunc = 'count', fill_value=0, margins=True)

In [87]:
# переименование колонок
pivot_table_family = pivot_table_family.rename(columns={'debt': 'debt', 0: 'no_debt', 1: 'debt', 'All': 'total_borrowers'})

In [88]:
# соединение таблицы с процентами со сводной таблицей методом merge() 
pivot_table_family_percent = pivot_table_family.merge(family_debt, on='family_status', how='right')

In [89]:
# вывод результатов
pivot_table_family_percent.sort_values(by='percentage_of_cases_of_debt', ascending=False)

,family_status,no_debt,debt,total_borrowers,percentage_of_cases_of_debt
4,не женат / не замужем,2536,274,2810,9.8
2,гражданский брак,3762,388,4150,9.3
3,женат / замужем,11408,931,12339,7.5
0,в разводе,1110,85,1195,7.1
1,вдовец / вдова,896,63,959,6.6


**Вывод**

Как и ожидалось у неженатых заемщиков процент невозврата кредита в срок составляет 9%, такую же картину мы можем наблюдать у заемщиков состоящих гражданском браке. Это связано с тем, что неженатые люди и состоящие в гражданском браке обычно рассчитывают только на свои силы, в то время как люди состоящие в браке чаще всего имеют семейный бюджет и в случае финансовых трудностей у одно из супругов могут покрыть расходы на погашение кредита и семейного бюджета. 

Самый низкий процент не возврата кредита у вдовцов / вдов - 6% случаев. Возможно данная категория заемщиков в качестве дополнительного обеспечения кредита имеет совместно нажитое имущество, доставшееся им после смерти одного из супругов. Также мы можем наблюдать, что на эту категорию приходится меньше всего заемщиков.

Дополнительно мы выяснили, что молодые люди чаще невозвращают кредиты, чем пенсионеры и взрослые люди.

Наибольшее количество заемщиков - это люди, состящие в браке. Они в 3 - 6 раз чаще берут кредиты, чем остальные категории заемщиков. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Выдвинем гипотезу, что люди с низким уровнем дохода чаще не возвращают кредиты не возвращают кредиты в срок. Для проверки гипотезы рассчитаем процент невозврата кредита по уровню дохода заемщиков.

In [90]:
# сохраним результат работы работы функции в переменной income_debt
income_debt = calc_percent(df, 'income_level', 'debt')

In [91]:
# создадим объект DataFrame с новым именем столбца
income_debt = income_debt.reset_index(name='percentage_of_cases_of_debt')

In [92]:
# переименуем столбцы
income_debt = income_debt.rename(columns={'index':'income_level', 
                                              'percentage_of_cases_of_debt':'percentage_of_cases_of_debt'})

In [93]:
# для наглядности округлим данные с точностью до одного знака после запятой
income_debt = income_debt.round(1)

Посмотрим на получившийся результат. Отсортируем в порядке убывания процентов.

In [94]:
income_debt.sort_values(by='percentage_of_cases_of_debt', ascending=False)

,income_level,percentage_of_cases_of_debt
0,высокий,20.0
3,низкий,8.1
2,ниже среднего,5.6
1,выше среднего,0.0


Для наглядности выведем сводную таблицу, в которой отразим количество должников и заемщиков по категориям и процент невозврата кредита. 

In [95]:
# создание сводной таблицы методом pivot_table()
pivot_table_income = df.pivot_table(index = 'income_level', columns = 'debt', values = 'children',
                                   aggfunc = 'count', fill_value=0, margins=True)

In [96]:
# переименование колонок
pivot_table_income = pivot_table_income.rename(columns={'debt': 'debt', 0: 'no_debt', 1: 'debt', 'All': 'total_borrowers'})

In [97]:
# соединение таблицы с процентами со сводной таблицей методом merge() 
pivot_table_income_percent = pivot_table_income.merge(income_debt, on='income_level', how='right')

In [98]:
# вывод результатов
pivot_table_income_percent.sort_values(by='percentage_of_cases_of_debt', ascending=False)

,income_level,no_debt,debt,total_borrowers,percentage_of_cases_of_debt
0,высокий,4,1,5,20.0
3,низкий,19580,1733,21313,8.1
2,ниже среднего,118,7,125,5.6
1,выше среднего,10,0,10,0.0


**Вывод**

Гипотеза не подтвердилась. Оказалось, больше всего невозвратов кредитов у людей с высоким уровнем дохода - 20%. Это может быть связано с тем, что люди с высоким уровнем дохода берут очень значительные суммы кредита, например на приобретение дорогостоящей недвижимости, в том числе в коммерческих целях. У больших сумм кредита большой риск невозврата, особенно если бизнес "прогорает". 


К группе наименьшего риска находятся люди с уровнем дохода "ниже среднего" и "выше среднего", что составляет 5% и 0% невозврата кредита соответственно.

Однако наибольшее число заемщиков - это люди с низким уровнем дохода, что в полне ожидаемо, так как у таких людей меньше возможностей приобретения различного рода ценностей без использования кредитных средств.

In [110]:
pivot_table_income_percent.style.set_caption('text')

,income_level,no_debt,debt,total_borrowers,percentage_of_cases_of_debt
0,высокий,4,1,5,20
1,выше среднего,10,0,10,0
2,ниже среднего,118,7,125,5.6
3,низкий,19580,1733,21313,8.1


- Как разные цели кредита влияют на его возврат в срок?

Выдвинем гипотезу, что заемщики, которые берут кредит на покупку недвижимости, чаще других не возвращают кредит в срок. Для проверки данного мнения рассчитаем процент невозврата кредита по целям кредита.

In [99]:
# сохраним результат работы работы функции в переменной purpose_debt
purpose_debt = calc_percent(df, 'purpose', 'debt')

In [100]:
# создадим объект DataFrame с новым именем столбца
purpose_debt = purpose_debt.reset_index(name='percentage_of_cases_of_debt')

In [101]:
# переименуем столбцы
purpose_debt = purpose_debt.rename(columns={'index':'purpose', 
                                              'percentage_of_cases_of_debt':'percentage_of_cases_of_debt'})

In [102]:
# для наглядности округлим данные с точностью до одного знака после запятой
purpose_debt = purpose_debt.round(1)

Посмотрим на получившийся результат. Отсортируем в порядке убывания процентов.

In [103]:
purpose_debt.sort_values(by='percentage_of_cases_of_debt', ascending=False)

,purpose,percentage_of_cases_of_debt
0,автомобиль,9.4
2,образование,9.2
3,свадьба,8.0
1,недвижимость,7.2


Для наглядности выведем сводную таблицу, в которой отразим количество должников и заемщиков по категориям и процент невозврата кредита. 

In [104]:
# создание сводной таблицы методом pivot_table()
pivot_table_purpose = df.pivot_table(index = 'purpose', columns = 'debt', values = 'children',
                                   aggfunc = 'count', fill_value=0, margins=True)

In [105]:
# переименование колонок
pivot_table_purpose = pivot_table_purpose.rename(columns={'debt': 'debt', 0: 'no_debt', 1: 'debt', 'All': 'total_borrowers'})

In [106]:
# соединение таблицы с процентами со сводной таблицей методом merge() 
pivot_table_purpose_percent = pivot_table_purpose.merge(purpose_debt, on='purpose', how='right')

In [107]:
# вывод результатов
pivot_table_purpose_percent.sort_values(by='percentage_of_cases_of_debt', ascending=False)

,purpose,no_debt,debt,total_borrowers,percentage_of_cases_of_debt
0,автомобиль,3903,403,4306,9.4
2,образование,3643,370,4013,9.2
3,свадьба,2138,186,2324,8.0
1,недвижимость,10028,782,10810,7.2


**Вывод**

Наша гипотеза не подтвердилась. Оказывается, что чаще всего не возвращают кредит, взятый на автомобиль или образование. Одним из факторов влияющих на сложность возврата кредита в срок, является то, что автомобиль сам по себе является источником незапланированных расходов, таких как: аварии и поломки, требующие ремонта. А вот образование приносит доход только в перспективе и человек окончивший обучение, может некоторое время оставаться безработным пока находятся в поисках новой работы.

Заемщики получившие кредит на недвижимость наиболее ответственно подходят к процессу возврата кредита в срок, так как невыплата кредита может грозить конфискацией приобретенной недвижимости. Также одним из способов погашения кредита является доход от сдачи приобретенной недвижимости в аренду.

Мы можем наблюдать, что наиболее востребован кредит на приобретение недвижимости.

## Шаг 4. Общий вывод

    Обобщая результаты исследования можно сделать следующие выводы: 
    1. заемщики не состоящие в браке, а также имеющие детей наиболее часто не могут вернуть кредит в срок
    2. люди с высоким уровнем дохода находится в группе высокого риска невозврата кредита
    3. кредиты, выданные на приобретение автомобиля и образование, чаще всего не возвращают в срок

Дополнительно нам удалось узнать, что заемщики в возрасте до 30 лет чаще невозвращают кредиты, чем заемщики в более старшем возрасте.

Выделим заемщиков в категории с высоким риском и низким риском невозврата кредита в срок.

In [108]:
# создание обобщающей таблицы
data = [
    ['есть дети', 'не женат / не замужем', 'высокий', 'автомобиль'],
    ['нет детей', 'вдовец / вдова', 'выше среднего', 'недвижимость'],    
]

columns = ['children_category', 'family_status', 'income_level', 'purpose']
final_data = pd.DataFrame(data=data , columns=columns, index=['высокий риск', 'низкий риск'])

In [109]:
# вывод результатов
final_data

,children_category,family_status,income_level,purpose
высокий риск,есть дети,не женат / не замужем,высокий,автомобиль
низкий риск,нет детей,вдовец / вдова,выше среднего,недвижимость
